## Voxel correlation exercise

In [1]:
# import common modules
import numpy as np  # the Python array package
import matplotlib.pyplot as plt  # the Python plotting package

import nibabel as nib

import nipraxis

Import the `events2neural` function from the Nipaxis module

In [2]:
from nipraxis.stimuli import events2neural

Fetch the image to work on.

In [3]:
# Load the function to fetch the data file we need.
# Fetch the data file.
data_fname = nipraxis.fetch_file('ds114_sub009_t2r1.nii')
# Show the file name of the fetched data.
data_fname

Load the image with Nibabel.

In [4]:
#- Load the ds114_sub009_t2r1.nii image
img = nib.load(data_fname)
# Show the result
img

In [5]:
assert img.affine.shape == (4, 4)
assert len(img.shape) == 4

In [6]:
#- Get the number of volumes in ds114_sub009_t2r1.nii
n_trs = img.shape[-1]
# Show the result
n_trs

The TR (time between scans) is 2.5 seconds.

In [7]:
#: TR
TR = 2.5

Get the condition file:

In [8]:
# Fetch the condition file
cond_fname = nipraxis.fetch_file('ds114_sub009_t2r1_cond.txt')
cond_fname

Call the `events2neural` function to give you a time course that is 1 for
the volumes during the task (thinking of verbs) and 0 for the volumes during
rest.

**Hint**: Check the help for `events2neural`, perhaps with Jupyter and the `?` suffix.

In [9]:
#- Call the events2neural function to generate the on-off values for
#- each volume.
time_course = events2neural(cond_fname, TR, n_trs)
# Plot the time course.
plt.plot(time_course)

Using slicing, drop the first *4* volumes, and the corresponding on-off values:

In [10]:
#- Drop the first 4 volumes, and the first 4 on-off values.
data = img.get_fdata()
data_from_4 = data[..., 4:]
time_course_from_4 = time_course[4:]

In [11]:
assert data_from_4.dtype == np.float64
assert data_from_4.shape[-1] == 169
assert time_course_from_4.shape == (169,)

Make a single brain-volume-sized array of all zero to hold the correlations:

In [12]:
#- Make a brain-volume-size array of 0 to hold the correlations
correlations = np.zeros(data.shape[:-1])

In [13]:
assert correlations.shape == img.shape[:-1]

* Loop over all voxel indices on the first, then second, then third dimension;
* extract the voxel time courses at each voxel coordinate in the image;
* get the correlation between the voxel time course and neural prediction;
* fill in the value in the correlations array.

For extra points - some voxels correlations will give a warning for the
calculation.  Why?  How can you fix this?

**Hint for extra points**: By default, you will only see a warning once, for
each cell you execute. If you want to be able to see each warning, for every
time the warning is valid, add the following code to your code cell below.

```python
import warnings
warnings.simplefilter("always")
```

**End of hint for extra points**.

In [14]:
#- Loop over all three voxel axes / indices.
#- Extract the voxel time courses at each voxel.
#- Get correlation value for voxel time course with on-off vector.
#- Fill value in the correlations array.
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            vox_values = data[i, j, k]
            if np.all(vox_values == 0):
                correlation = np.nan
            else:
                correlation = np.corrcoef(time_course, vox_values)[1, 0]
            correlations[i, j, k] = correlation

In [15]:
# Check you get the same answer when selecting a voxel time course
# and running the correlation on that time course.  One example voxel
# could be the voxel at array coordinate [42, 32, 19]
voxel_time_course = data[42, 32, 19]
single_cc = np.corrcoef(voxel_time_course, time_course)[0, 1]
assert np.allclose(correlations[42, 32, 19], single_cc)

Plot the middle plane (slice on the third axis) from the correlations array.
Can you see any sign of activity (high correlation) in the frontal lobe?

In [16]:
#- Plot the middle slice of the third axis from the correlations array
plt.imshow(correlations[:, :, 14], cmap='gray')